<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Leveraging Apache Spark for Smart Building HVAC Monitoring

**Estimated time needed: 30 minutes**

### Objectives

After completing this lab, you will be able to:

- Explain the distributed architecture of Spark in the context of smart building monitoring
- Simulate real-time sensor data for HVAC systems in a building
- Perform SQL queries to detect critical environmental conditions and calculate average readings
- Determine the aggregated results to the console for immediate insights into room conditions


## Background
Smart Building Solutions, Inc. specializes in optimizing HVAC (heating, ventilation, and air conditioning) systems to enhance comfort and energy efficiency in commercial buildings. By monitoring temperature and humidity levels in real-time across various rooms, the company aims to ensure optimal indoor conditions and preemptively address potential HVAC issues.

With a continuous influx of sensor data, Smart Building Solutions needs to process and analyze this data in real-time to maintain the quality of the indoor environment.

## Data set description
The simulated data set comprises:

`room_id`: Unique identifier for each room (e.g., R001, R002).

`temperature`: Current temperature reading from the sensor (in °C).

`humidity`: Current humidity level reading from the sensor (in %).

`timestamp`: Time when the reading was recorded (automatically generated by Spark).
The data is generated at a rate of 5 rows per second, simulating multiple rooms with various environmental conditions.


## Challenges
Monitoring indoor environmental conditions poses several challenges:

**High data velocity**: Continuous data from multiple sensors can overwhelm traditional systems.

**Need for immediate alerts**: Delays in identifying critical conditions can lead to discomfort or system inefficiencies.

**Need for data aggregation and analysis**: Efficiently aggregating and analyzing real-time data for proactive maintenance and optimization is essential.

## Apache Spark with structured streaming
To address these challenges, Apache Spark is employed for its powerful distributed computing capabilities, enabling real-time data processing and analytics.


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.clustering import KMeans


from pyspark.sql import SparkSession


### Set up the Spark session:


In [3]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Smart Building HVAC Monitoring") \
    .getOrCreate()


25/06/23 06:42:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Simulate sensor data:

Use Spark’s rate source to generate continuous readings from multiple rooms.


In [4]:
from pyspark.sql.functions import expr, rand,when

# Simulate sensor data with room IDs and readings
sensor_data = spark.readStream.format("rate").option("rowsPerSecond", 5).load() \
    .withColumn("room_id", expr("CAST(value % 10 AS STRING)")) \
    .withColumn("temperature", when(expr("value % 10 == 0"), 15)  # Set temperature to 15 for one specific record
                .otherwise(20 + rand() * 25)) \
    .withColumn("humidity", expr("40 + rand() * 30"))

### Create a temporary SQL view:

Create temporary SQL view to perform SQL queries on the streaming data.


In [5]:
# Create a temporary SQL view for the sensor data
sensor_data.createOrReplaceTempView("sensor_table")


### Define SQL queries for aggregation and analysis:

* **Critical temperature query**: Detect rooms with critical temperature levels
* **Average readings query**: Calculate average readings over a 1-minute window
* **Attention needed query**: Identify rooms that need immediate attention based on humidity levels


In [6]:
# SQL Query to detect rooms with critical temperatures
critical_temperature_query = """
    SELECT 
        room_id, 
        temperature, 
        humidity, 
        timestamp 
    FROM sensor_table 
    WHERE temperature < 18 OR temperature > 60
"""

# SQL Query to calculate average readings over a 1-minute window
average_readings_query = """
    SELECT 
        room_id, 
        AVG(temperature) AS avg_temperature, 
        AVG(humidity) AS avg_humidity, 
        window.start AS window_start 
    FROM sensor_table
    GROUP BY room_id, window(timestamp, '1 minute')
"""

# SQL Query to find rooms that need immediate attention based on humidity
attention_needed_query = """
    SELECT 
        room_id, 
        COUNT(*) AS critical_readings 
    FROM sensor_table 
    WHERE humidity < 45 OR humidity > 75
    GROUP BY room_id
"""


### Execute the SQL queries:

Execute each SQL query to create streaming DataFrames.


In [7]:
# Execute the critical temperature query
critical_temperatures_stream = spark.sql(critical_temperature_query)


# Execute the average readings query
average_readings_stream = spark.sql(average_readings_query)

# Execute the attention needed query
attention_needed_stream = spark.sql(attention_needed_query)






### Output the results to the console:

Display the results from each query in real-time.


In [8]:
# Output the results to the console for all queries
critical_query = critical_temperatures_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .queryName("Critical Temperatures") \
    .start()

average_query = average_readings_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Average Readings") \
    .start()

attention_query = attention_needed_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Attention Needed") \
    .start()



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------+--------+---------+
|room_id|temperature|humidity|timestamp|
+-------+-----------+--------+---------+
+-------+-----------+--------+---------+



[Stage 0:>                                                          (0 + 1) / 8]

### Keep the streams running:

Ensure that the streaming queries continue to run to process incoming data.


In [9]:
# Keep the streams running

print("********Critical Temperature Values*******")
critical_query.awaitTermination()

print("********Average Readings Values********")
average_query.awaitTermination()
print("********Attention Needed Values********")
attention_query.awaitTermination()


[Stage 0:>                                                          (0 + 8) / 8]

********Critical Temperature Values*******


-------------------------------------------
Batch: 1
-------------------------------------------


[Stage 2:>                (8 + 8) / 200][Stage 4:>                (0 + 0) / 200]

+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 62.15234753158954|2025-06-23 06:45:...|
|      0|       15.0| 65.85280091327557|2025-06-23 06:45:...|
|      0|       15.0|40.036971629645194|2025-06-23 06:45:...|
|      0|       15.0|47.051201908570796|2025-06-23 06:45:...|
+-------+-----------+------------------+--------------------+



...

+-------+-----------+-----------------+--------------------+
|room_id|temperature|         humidity|           timestamp|
+-------+-----------+-----------------+--------------------+
|      0|       15.0|65.05092386342399|2025-06-23 06:45:...|
|      0|       15.0|68.75995393474045|2025-06-23 06:45:...|
|      0|       15.0|65.79080961227874|2025-06-23 06:45:...|
|      0|       15.0| 46.9459126727938|2025-06-23 06:45:...|
|      0|       15.0|65.43824176782947|2025-06-23 06:45:...|
+-------+-----------+-----------------+--------------------+

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+---------------+------------+------------+
|room_id|avg_temperature|avg_humidity|window_start|
+-------+---------------+------------+------------+
+-------+---------------+------------+------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|43.410447519462636|2025-06-23 06:45:...|
|      0|       15.0| 52.82934950696093|2025-06-23 06:46:...|
|      0|       15.0| 55.39403401751639|2025-06-23 06:46:...|
|      0|       15.0|61.681853726900705|2025-06-23 06:46:...|
|      0|       15.0| 69.65635647254368|2025-06-23 06:46:...|
|      0|       15.0| 51.57893674097831|2025-06-23 06:46:...|
|      0|       15.0| 52.24156854853378|2025-06-23 06:45:...|
|      0|       15.0|60.066426947360284|2025-06-23 06:46:...|
|      0|       15.0| 47.28557781369107|2025-06-23 06:46:...|
|      0|       15.0|53.314803848189264|2025-06-23 06:46:...|
|      0|       15.0| 50.57794679215662|2025-06-23 06:46:...|
|      0|       15.0| 53.7405982165

[Stage 10:>(66 + 8) / 200][Stage 11:>   (0 + 0) / 8][Stage 12:>   (0 + 0) / 8]8]

KeyboardInterrupt: 

[Stage 10:(163 + 8) / 200][Stage 11:>   (0 + 0) / 8][Stage 12:>   (0 + 0) / 8]

### Conclusion
In this lab, you explored the use of Apache Spark in smart building monitoring, particularly focusing on HVAC (heating, ventilation, and air conditioning) systems. You now understand the Spark's distributed architecture. You also understand how to simulate real-time sensor data for temperature and humidity, execute SQL queries to identify critical environmental conditions, and output aggregated results for immediate insights.


## Author(s)

Lakshmi Holla

## Other Contributors
Malika Singla
